In [15]:
from xgboost import plot_importance
from xgboost import XGBRegressor
from xgboost import XGBClassifier

import pandas as pd
import numpy as np

In [16]:
sales = pd.read_csv('./sales_train.csv')
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [24]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_month  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [17]:
item_cnt_month = sales.groupby(by='date_block_num').sum()['item_cnt_day']

In [46]:
item_cnt_month = pd.DataFrame(item_cnt_month)
item_cnt_month.head(3)

,item_cnt_day
date_block_num,
0,131479.0
1,128090.0
2,147142.0


In [51]:
item_cnt_month.tail()

,item_cnt_day
date_block_num,
29,64114.0
30,63187.0
31,66079.0
32,72843.0
33,71056.0


In [65]:
sales = pd.merge(sales, item_cnt_month, how='left', left_on='date_block_num', right_on='date_block_num')

In [66]:
sales.head(50)

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y,item_cnt_day
0,0,59,22154,999.00,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
1,0,25,2552,899.00,128090.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
2,0,25,2552,899.00,147142.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
3,0,25,2554,1709.05,107190.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
4,0,25,2555,1099.00,106970.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
5,0,25,2564,349.00,125381.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
6,0,25,2565,549.00,116966.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
7,0,25,2572,239.00,125291.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
8,0,25,2572,299.00,133332.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0
9,0,25,2573,299.00,127541.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0,131479.0


In [20]:
sales.drop('item_cnt_day', axis=1, inplace=True)

In [25]:
sales.drop('date', axis=1, inplace=True)

In [26]:
sales.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,0,59,22154,999.00,131479.0
1,0,25,2552,899.00,128090.0
2,0,25,2552,899.00,147142.0
3,0,25,2554,1709.05,107190.0
4,0,25,2555,1099.00,106970.0


In [37]:
len(sales)

2935849

In [36]:
len(sales[sales.item_cnt_month.isnull()])

2935815

In [29]:
x = sales.iloc[:,:-1]
y = sales.iloc[:,-1]
x

,date_block_num,shop_id,item_id,item_price
0,0,59,22154,999.00
1,0,25,2552,899.00
2,0,25,2552,899.00
3,0,25,2554,1709.05
4,0,25,2555,1099.00
...,...,...,...,...
2935844,33,25,7409,299.00
2935845,33,25,7460,299.00
2935846,33,25,7459,349.00
2935847,33,25,7440,299.00


In [30]:
y

0          131479.0
1          128090.0
2          147142.0
3          107190.0
4          106970.0
             ...   
2935844         NaN
2935845         NaN
2935846         NaN
2935847         NaN
2935848         NaN
Name: item_cnt_month, Length: 2935849, dtype: float64

In [ ]:
xgb = XGBRegressor()